# 逻辑回归
构建逻辑回归模型来预测学生能否被大学录取
## 样本数据可视化
训练样本存储在ex2data1.txt文件中，包括申请人两门测试的分数和录用结果，画出样本的散点图

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np

# 假设 data 是之前 read_csv 读取的 DataFrame
# names=['Exam1', 'Exam2', 'Admitted']
def plotData(data):
    # 1. 筛选数据：将样本分为 正例(1) 和 负例(0)
    positive = data[data['Admitted'] == 1]
    negative = data[data['Admitted'] == 0]

    plt.figure(figsize=(10, 6))

    # 2. 绘制散点图
    # s 为点的大小，c 为颜色，marker 为形状
    plt.scatter(positive['Exam1'], positive['Exam2'], 
                s=50, c='b', marker='+', label='Admitted')
    
    plt.scatter(negative['Exam1'], negative['Exam2'], 
                s=50, c='r', marker='o', label='Not Admitted')

    # 3. 装饰图表
    plt.xlabel('Exam 1 Score')
    plt.ylabel('Exam 2 Score')
    plt.legend() # 显示图例，区分 Admitted/Not Admitted
    plt.title('Scatter Plot of Training Data')
    plt.show()

if __name__ == "__main__":
    path = "ex2data1.txt"
    data = pd.read_csv(path, header=None, names=['Exam1', 'Exam2', 'Admitted'])
    print(f"样本数据：\n{data.head()}")
    plotData(data)

## sigmoid函数
逻辑回归的假设函数表示为
$$h_{\theta}(x)=g(\theta^Tx)$$
$$g(z)=\frac{1}{1+e^{-z}}$$

In [ ]:
# 定义 Sigmoid 函数
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

if __name__ == "__main__":
    # 生成自变量数据：从 -10 到 10，生成 100 个点
    z = np.linspace(-10, 10, 100)
    g = sigmoid(z)

    # 绘图
    plt.figure(figsize=(8, 5))
    plt.plot(z, g, 'b-', linewidth=2, label=r'$g(z) = \frac{1}{1 + e^{-z}}$')

    # 装饰图表
    plt.axhline(y=0.5, color='r', linestyle='--', linewidth=1, label='Threshold (0.5)') # 画出 0.5 决策线
    plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3) # 画出 y 轴

    plt.title("Sigmoid Function", fontsize=14)
    plt.xlabel("z")
    plt.ylabel("g(z)")
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)

    plt.show()

## 代价函数和梯度下降
使用梯度下降算法求解使代价函数最小时的$\theta$值。逻辑回归的代价函数表示为
$$J(\theta)=\frac{1}{m}\sum_{i=1}^m[-y^{(i)}\log (h_{\theta}(x^{(i)}))-(1-y^{(i)})\log (1-h_{\theta}(x^{(i)}))]$$
代价函数偏导数表示为
$$\frac{\partial J(\theta)}{\partial \theta_j}=\frac{1}{m}\sum_{i=1}^m(h_{\theta}(x^{(i)})-y^{(i)})x_j^{(i)}\quad(j=0,1,...,n)$$
梯度下降算法：
$$\begin{align*}
Repeat:\{\theta_j&=\theta_j-\alpha\frac{\partial}{\partial \theta_j}J(\theta)\\
&=\theta_j-\frac{\alpha}{m}\sum_{i=1}^m(h_{\theta}(x^{(i)})-y^{(i)})x_j^{(i)}\quad(j=0,1,...,n)\\
&(同时更新所有\theta)\}
\end{align*}$$
其中$m$为样本数量，$n$为特征数量。需要注意的是虽然逻辑回归的代价函数偏导数形式和线性回归相同，但$h_{\theta}(x)$的定义完全不同

In [ ]:
# 对于 m 条包含 n 个特征的样本（矩阵形状为 (m,n)，归一化的标准流程是按列计算
def featureNormalization(X):
    mu = np.mean(X, axis=0) # 计算每列的均值
    sigma = np.std(X, axis=0) # 计算每列的标准差
    sigma[sigma == 0] = 1 # 避免除以 0（针对特征值全部相同的列）
    X = (X - mu) / sigma
    print(X.head())
    return X, 

def addOnes(X):
    ones = np.ones((len(X), 1))
    return np.hstack((ones, X))

def costFunction(theta, X, y):
    theta = theta.reshape(-1, 1) # 确保theta为列向量

    h = sigmoid(X @ theta)
    eps = 1e-15 # 添加 eps (1e-15) 防止 np.log(0) 导致溢出

    term1 = np.log(h + eps) * y
    term2 = np.log(1 - h + eps) * (1 - y)
    return -np.mean(term1 + term2)
    # 另一种矩阵乘法形式的写法：return - 1 / m * (y.T @ np.log(h) + (1 - y.T) @ np.log(1 - h)).item()

def grad(X, y, theta):
    theta = theta.reshape(-1, 1)

    h = sigmoid(X @ theta)
    error = h - y
    return 1 / len(X) * (X.T @ error)

def gradientDescent(X, y, theta, alpha, iters):
    cost = np.zeros(iters + 1)

    for iter in range(iters):
        cost[iter] = costFunction(theta, X, y)
        theta -= alpha * grad(X, y, theta)
    cost[iter+1] = costFunction(theta, X, y)
    return theta, cost

if __name__ == "__main__":
    # 读取DataFrame格式的特征和标签数据
    cols = data.shape[1]
    X = data.iloc[:, :-1]
    y = data.iloc[:, cols-1:cols]
    # 特征归一化
    X = featureNormalization(X)

    # 特征和标签转换为ndarray类型的二维矩阵
    X = X.to_numpy()
    y = y.to_numpy()
    # X添加theta_0对应列
    X = addOnes(X)
    print(f"X:shape{X.shape}\n{X[:5, :]}")
    print(f"y:shape{y.shape}\n{y[:5, :]}")

    # 参数初始化
    initial_theta = np.zeros((X.shape[1], 1), dtype=np.float32)
    print(f"theta:shape{initial_theta.shape}\n{initial_theta}")

    # 计算初始代价函数值，验证代价函数计算准确性
    print(f"initial cost:{costFunction(initial_theta, X, y):.3f}") # initial cost:0.693

    alphas = np.array([0.01, 0.03, 0.1, 0.3])
    iters = 1000

    plt.figure(figsize=(12,8))

    for alpha in alphas:
        current_theta = initial_theta.copy() # 每次循环都使用初始 theta，防止上一个 alpha 的结果干扰
        theta, cost = gradientDescent(X, y, current_theta, alpha, iters)
        print(f"theta for alpha = {alpha}:{theta}")
        print(f"final cost for alpha = {alpha}:{cost[-1]:.3f}")
        plt.plot(range(len(cost)), cost, linewidth=2, label=f"alpha={alpha}")
    
    plt.title("Convergence of Cost Function for different Alphas")
    plt.xlabel("Iterations")
    plt.ylabel("Cost J")
    plt.legend() # 显示不同 alpha 的标签
    plt.grid(True)
    plt.show() # 最后统一显示


## 使用库函数计算$\theta$的值
使用scipy.optimize.fmin_tnc计算$\theta$

In [ ]:
import scipy.optimize as opt

#  fmin_tnc 要求梯度返回必须是一维数组
def gradOneDimension(theta, X, y): # 注意：fmin_tnc使用的cost和grad函数入参第一个必须是theta，其他入参顺序和args相同
    theta = theta.reshape(-1, 1)

    h = sigmoid(X @ theta)
    error = h - y
    return (1 / len(X) * (X.T @ error)).flatten()

if __name__ == "__main__":
    initial_theta_1 = np.zeros(X.shape[1])
    print(f"initial_theta_1:shape{initial_theta_1.shape}")
    result = opt.fmin_tnc(func=costFunction, x0=initial_theta_1, fprime=gradOneDimension, args=(X, y)) # func:待最小化的目标函数；x0:参数的初始值，必须为一维数组；fprime:目标函数的梯度函数；args:传递给目标函数和梯度函数的额外参数
    print(f"result returned by fmin_tnc:\n{result}")
    print(f"final cost for theta from fmin_tnc:{costFunction(result[0], X, y):.3f}") # final cost for theta from fmin_tnc:0.203

## 画决策边界
决策边界表示为$\theta^Tx=0$

In [ ]:
def plotDecisionBoundary(data, theta):
    plt.figure(figsize=(10, 6))

    # 1. 筛选数据：将样本分为 正例(1) 和 负例(0)
    positive = data[data['Admitted'] == 1]
    negative = data[data['Admitted'] == 0]

    # 2. 绘制散点图
    # s 为点的大小，c 为颜色，marker 为形状
    plt.scatter(positive['Exam1'], positive['Exam2'], 
                s=50, c='b', marker='+', label='Admitted')
    
    plt.scatter(negative['Exam1'], negative['Exam2'], 
                s=50, c='r', marker='o', label='Not Admitted')

    # 3. 画决策边界
    exam1score_uniform_raw = np.linspace(data['Exam1'].min(), data['Exam1'].max(), 100)
    exam1score_uniform_scaled = featureNormalization(exam1score_uniform_raw)
    exam2score_uniform_scaled = (- theta[0] - theta[1] * exam1score_uniform_scaled) / theta[2]
    exam2score_uniform_raw = exam2score_uniform_scaled * exam1score_uniform_raw.std() + exam1score_uniform_raw.mean()
    plt.plot(exam1score_uniform_raw, exam2score_uniform_raw, 'g-', label='Decision Boundary')

    # 4. 装饰图表
    plt.xlabel('Exam 1 Score')
    plt.ylabel('Exam 2 Score')
    plt.legend() # 显示图例，区分 Admitted/Not Admitted
    plt.show()

if __name__ == "__main__":
    plotDecisionBoundary(data, theta.flatten())
    plotDecisionBoundary(data, result[0].flatten())

## 模型评估

In [ ]:
def predict(theta, X):
    probability = sigmoid(X @ theta)
    return [1 if p >= 0.5 else 0 for p in probability]

if __name__ == "__main__":
    # 使用新样本评估
    samp = np.array([45, 85]).reshape(1, -1)
    samp = addOnes(samp)
    samp_scaled = featureNormalization(samp)
    print(f"Predicted Admission Probability:{sigmoid(samp_scaled @ theta).flatten()}")

    # 使用训练集评估模型准确度
    pred = predict(theta, X)
    correct = [1 if (a == b) else 0 for (a, b) in zip(pred, y)]
    accuracy = sum(correct) % len(correct)
    print(f"accuracy:{accuracy}%") # accuracy:89%